In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

/home/dcbowma2/cs240/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mini = pd.read_csv('mini.csv')
print(mini)

    patent_id section_id subsection_id  group_id       subgroup_id  \
0     6837198      ['Y']       ['Y10']  ['Y10T']  ['Y10T74/18288']   
1     6837201      ['F']       ['F01']  ['F01M']     ['F01M9/106']   
2     6837214      ['Y']       ['Y02']  ['Y02T']     ['Y02T10/44']   
3     6837226      ['F']       ['F02']  ['F02M']     ['F02M26/48']   
4     6840170      ['Y']       ['Y10']  ['Y10T']  ['Y10T74/18296']   
5     6840171      ['B']       ['B65']  ['B65H']  ['B65H2301/522']   
6     6840186      ['F']       ['F16']  ['F16G']     ['F16G11/14']   
7     6840191      ['A']       ['A01']  ['A01K']    ['A01K5/0128']   
8     6840401      ['B']       ['B65']  ['B65D']   ['B65D83/0805']   
9     6845797      ['C']       ['C08']  ['C08L']     ['C08L21/00']   
10    6845805      ['F']       ['F21']  ['F21S']     ['F21S11/00']   
11    6845808      ['B']       ['B22']  ['B22D']     ['B22D27/00']   
12    6846012      ['B']       ['B60']  ['B60N']      ['B60N2/28']   
13    6848340      [

In [9]:
from tokenize import group


def group_dist(s1,s2):
    section1,section2 = s1[2],s2[2]
    subsection1,subsection2 = s1[3:5],s2[3:5]
    group1,group2 = s1[5],s2[5]
    print(section1)

    if section1 != section2:
        return 64
    if subsection1 != subsection2:
        return 16
    if group1 != group2:
        return 4
    return 0

for x in mini['group_id']:
    for y in mini['group_id']:
        print(f'{x}, {y}')
        print(group_dist(x,y))

['Y10T'], ['Y10T']
Y
0
['Y10T'], ['F01M']
Y
64
['Y10T'], ['Y02T']
Y
16
['Y10T'], ['F02M']
Y
64
['Y10T'], ['Y10T']
Y
0
['Y10T'], ['B65H']
Y
64
['Y10T'], ['F16G']
Y
64
['Y10T'], ['A01K']
Y
64
['Y10T'], ['B65D']
Y
64
['Y10T'], ['C08L']
Y
64
['Y10T'], ['F21S']
Y
64
['Y10T'], ['B22D']
Y
64
['Y10T'], ['B60N']
Y
64
['Y10T'], ['B67B']
Y
64
['Y10T'], ['A23G']
Y
64
['Y10T'], ['F16H']
Y
64
['Y10T'], ['B62D']
Y
64
['Y10T'], ['A63B']
Y
64
['Y10T'], ['B43L']
Y
64
['F01M'], ['Y10T']
F
64
['F01M'], ['F01M']
F
0
['F01M'], ['Y02T']
F
64
['F01M'], ['F02M']
F
16
['F01M'], ['Y10T']
F
64
['F01M'], ['B65H']
F
64
['F01M'], ['F16G']
F
16
['F01M'], ['A01K']
F
64
['F01M'], ['B65D']
F
64
['F01M'], ['C08L']
F
64
['F01M'], ['F21S']
F
16
['F01M'], ['B22D']
F
64
['F01M'], ['B60N']
F
64
['F01M'], ['B67B']
F
64
['F01M'], ['A23G']
F
64
['F01M'], ['F16H']
F
16
['F01M'], ['B62D']
F
64
['F01M'], ['A63B']
F
64
['F01M'], ['B43L']
F
64
['Y02T'], ['Y10T']
Y
16
['Y02T'], ['F01M']
Y
64
['Y02T'], ['Y02T']
Y
0
['Y02T'], ['F02M']
Y

### Sort the data

In [10]:
top_level= {}
mid_level = {}
group_level = {}

for label in mini.iterrows():
    tl = label[1]['group_id'][2]
    ml = label[1]['group_id'][3:5]
    gl = label[1]['group_id'][5]
    sentence = label[1]['text']
    
    if tl in top_level:
        top_level[tl].append(sentence)
    else:
        top_level[tl] = [sentence]
    
    if ml in mid_level:
        mid_level[ml].append(sentence)
    else:
        mid_level[ml] = [sentence]

    if gl in group_level:
        group_level[gl].append(sentence)
    else:
        group_level[gl] = [sentence]

In [5]:
[group_level[t][0][0:20] for t in group_level]

['A variable valve tra',
 'An intake valve for ',
 'An apparatus for thr',
 'A cleat comprising: ',
 'A device for receivi',
 'A baffle structure f',
 'A tire comprising: a',
 'Light guiding blinds',
 'A sensor assembly fo',
 'A structural improve']

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = mini['text']
groups = mini['group_id']

train_examples = [InputExample(texts=group_level[t], label=t) for t in group_level]

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

#Sentences are encoded by calling model.encode()
# embeddings = model.encode(sentences)

# #Print the embeddings
# for group,sentence, embedding in zip(groups,sentences, embeddings):
#     print("Sentence:", sentence[1:35], "Group:", group)
#     print(group_dist(groups[0],group))
#     print(np.dot(embedding,embeddings[0])/(np.linalg.norm(embedding)*np.linalg.norm(embeddings[0])))

model

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]


IndexError: list index out of range